In [2]:
import numpy as np

In [ ]:
class LoLiMoT:
    def __init__(self, input_dim, num_neurons=5):
        self.num_neurons = num_neurons
        self.input_dim = input_dim
        self.centers = np.random.uniform(-1, 1, size=(num_neurons, input_dim))  
        self.variances = np.ones((num_neurons, input_dim)) 
        self.weights = [np.random.randn(input_dim + 1) for _ in range(num_neurons)]  

    def gaussian_kernel(self, X, center, variance):
        return np.exp(-0.5 * np.sum(((X - center) ** 2) / variance, axis=1))

    def predict(self, X):
        n_samples = X.shape[0]
        outputs = np.zeros((n_samples, self.num_neurons))
        kernel_weights = np.zeros((n_samples, self.num_neurons))

        for i in range(self.num_neurons):
            kernel_weights[:, i] = self.gaussian_kernel(X, self.centers[i], self.variances[i])
            outputs[:, i] = np.dot(np.c_[X, np.ones(n_samples)], self.weights[i])

        kernel_weights /= (np.sum(kernel_weights, axis=1, keepdims=True) + 1e-8)

        final_output = np.sum(kernel_weights * outputs, axis=1)
        return final_output, kernel_weights


    def compute_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)


    def update_parameters(self, X, y, y_pred, kernel_weights, learning_rate=0.01):
        n_samples, n_features = X.shape

        for i in range(self.num_neurons):
            local_error = (y - y_pred) * kernel_weights[:, i]
            X_with_bias = np.c_[X, np.ones(n_samples)]
            gradient = -2 * np.dot(local_error, X_with_bias) / n_samples
            
            self.weights[i] -= learning_rate * gradient


        for i in range(self.num_neurons):
            center_grad = np.sum(kernel_weights[:, i:i+1] * (X - self.centers[i]), axis=0)
            self.centers[i] += learning_rate * center_grad

        
        for i in range(self.num_neurons):
            variance_grad = np.sum(kernel_weights[:, i:i+1] * ((X - self.centers[i]) ** 2), axis=0)
            self.variances[i] += learning_rate * variance_grad


    def train(self, X, y, epochs=100, learning_rate=0.01):
        for epoch in range(epochs):
            y_pred, kernel_weights = self.predict(X)
            loss = self.compute_loss(y, y_pred)

            self.update_parameters(X, y, y_pred, kernel_weights, learning_rate)

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

        return loss

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = housing.data
y = housing.target

X_normalized = scaler_X.fit_transform(X)
y_normalized = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

train_size = int(0.8 * len(X_normalized))
X_train, X_test = X_normalized[:train_size], X_normalized[train_size:]
y_train, y_test = y_normalized[:train_size], y_normalized[train_size:]


NN = LoLiMoT(input_dim=8, num_neurons=5)  
NN.train(X_train, y_train, epochs=100, learning_rate=0.0001)
y_pred, _ = NN.predict(X_test)

y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).ravel()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()


def MSE(y_true, y_pred):
    return np.mean((y_test_original - y_pred_original) ** 2)

mse = MSE(y, y_pred)
print(f"Mean Squared Error: {mse}")

Epoch 0, Loss: 4.6652
Epoch 10, Loss: 3.7959
Epoch 20, Loss: 3.0852
Epoch 30, Loss: 2.9844
Epoch 40, Loss: 3.0336
Epoch 50, Loss: 3.1020
Epoch 60, Loss: 3.0964
Epoch 70, Loss: 3.1800
Epoch 80, Loss: 3.1628
Epoch 90, Loss: 3.1383
Mean Squared Error: 4.197580260634937
